# **Python for Finance -- Session 2 -- Assignment**
---
<img src="http://www.doc.ic.ac.uk/~afd/images/logo_imperial_college_london.png" align = "left" width=200>
 <br><br><br><br>
 
- Copyright (c) Jack Jacquier, 2021. All rights reserved

- Author: Antoine Jacquier <a.jacquier@imperial.ac.uk>

- Platform: Tested on Windows 10 with Python 3.9

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad, simps

## Discrete versus continuous compounding¶

Suppose that you invest $\Pi$ GBP today in a risk-free bank account.
Considering compound interest rate yields the amount if one is earning  is
\begin{equation*}
\Pi\left(1+\frac{r}{n}\right)^{nT}.
\end{equation*}
over the period $[0,T]$, where $r$ is the **annual nominal interest rate** and $n$ the number of times the interest is compounded per year.
For example, if $n=1$, this is a yearly rate, for $n=2$, this is a 6-month rate, and $n=252$ corresponds to a daily rate.

**Question:** 
- Write a program computing  how much money will be compounded as a function of $\Pi, r, n, T$.
- As $n$ becomes large, compute the limit of Equation (1), and illustrate it numerically.

Available data:
- Annual rates provided by the Bank of England are available here: https://www.bankofengland.co.uk/boeapps/database/Bank-Rate.asp

- Annual rates by Bank of China can be found here: https://www.global-rates.com/en/interest-rates/central-banks/central-bank-china/pbc-interest-rate.aspx

In [3]:
def compound_interest_calculator(Pi, n, r, T):
    """
    INPUT:
    Pi: GBP invested today in a risk-free bank account
    n: number of times the interest is compounded per year
    r: annual interest rate
    T: maturity

    OUTPUT: 
    how much money will be compounded
    """
    total = Pi * (1+r/n)**(n*T)
    return (total - Pi)

In [4]:
Pi = 100 #GBP invested today in a risk-free bank account
n = 2 #number of times the interest is compounded per year
r = 0.01 * 1.75 #annual interest rate
T = 10 #maturity
print("money compounded: ",compound_interest_calculator(Pi, n, r, T))

money compounded:  19.033979935610674


## Pricing a Call option in Black-Scholes

We will consider the following values for the parameters:
$$
(S_0, K, \sigma, t, T) = (100, 100, 20\%, 0, 1 \text{ year}).
$$

### Pricing by integration

\begin{equation*}
C(S_0, K, T) = \mathbb{E}\left[\max(S_T-K, 0)\right] = 
\mathbb{E}\left[(S_{T}-K)\mathbb{1}_{S_T\geq K} + 0\cdot\mathbb{1}_{S_T< K}\right].
\end{equation*}
where
$$
S_T
 = S_0\exp\left\{-\frac{\sigma^2 T}{2} + \sigma W_T\right\}
 = S_0\exp\left\{-\frac{\sigma^2 T}{2} + \sigma\widetilde{n}\sqrt{T}\right\} \text{ (in distribution)},
$$
*Note: We consider no interest rate nor dividends here*.

Write a Python function pricing a Call option in the Black-Scholes model by integration. 
Check that with the parameters above you obtain a Call option price equal to $7.965567455405798$.

In [21]:
"""
max(S_T - K, 0) is essentially a function of a standard normal variable \tilde{n}
"""

def call_payoff(x, S0, K, T, sgm):
    ST = S0*np.exp(-(sgm**2*T)/2 + sgm*np.sqrt(T)*x)
    if(ST-K>0):
        return ST-K
    else:
        return 0

def standard_gauss_density(x):
    return np.exp(-x*x / (2.)) / (np.sqrt(2.*np.pi))

def integrand(x, S0, K, T, sgm):
    return call_payoff(x, S0, K, T, sgm)*standard_gauss_density(x)

def BS_pricing_integrate(S0, K, sgm, t, T):
    """
    INPUT: 
    S0: intial stock price (at t=0)
    K: strike price
    sgm: (constant) volatility
    t: the time when a call is valued
    T: maturity

    OUTPUT:
    c = price of a European call given by Black-Scholes model
    """
    xMin = -100 # interval for integration
    xMax = -xMin
    c_quad = quad(integrand, xMin, xMax, args=(S0, K, T, sgm,)) # quadrature
    
    x_vec = np.linspace(xMin, xMax, 10000) # Simpsons
    y_vec = [integrand(x, S0, K, T, sgm) for x in x_vec]
    c_simp = simps(y_vec, x_vec)
    # return c_quad
    return c_simp


In [26]:
S0 = 100
K = 100
sgm = 20 * 0.01
t = 0
T = 1
print("call price = ",BS_pricing_integrate(S0, K, sgm, t, T)[0])

call price =  7.965567459675685


### Pricing by simulation

Using the representation
\begin{equation*}
S_T
 = S_0\exp\left\{-\frac{\sigma^2 T}{2} + \sigma W_T\right\}
 = S_0\exp\left\{-\frac{\sigma^2 T}{2} + \sigma\widetilde{n}\sqrt{T}\right\} \text{ (in distribution)},
\end{equation*}
and the approximation
$$
\mathbb{E}[f(S_T)] \approx \frac{1}{N}\sum_{i=1}^{N}f\left(S_T^{(i)}\right),
$$
for any continuous function $f$, where $(S_T^{(i)})_{i=1,\ldots,N}$ are independent random copies of $S_T$, write a Python function for a Call option price. With the same parameters as in 2.1, plot the convergence of the Call price as a function of $N$.

**Remark:** Note that here, you do not in fact need to simulate trajectories of the Brownian motion since the option is *European*, i.e. only depends on the value of the underlying stock price at maturity. For path-dependent options, however, simulations of the paths are required.

## Implied volatility

Recall that in the Black-Scholes model (without interest rates), the value of a European Call option on $(S_t)_{t\geq 0}$ is given at inception by
$$
C^{\mathrm{BS}}(S_0, K, T;\sigma) = S_0\left(\mathcal{N}(d_{+}) - \mathrm{e}^{k}\mathcal{N}(d_{-})\right),
$$
where
$$
d_{\pm} = \frac{-k}{\sigma\sqrt{T}} \pm\frac{\sigma\sqrt{T}}{2}
\quad \text{ and } \quad k := \log\left(\frac{K}{S_0}\right),
$$
and where $\mathcal{N}$ denotes the Gaussian cumulative distribution function.


- For any fixed $k\in\mathbb{R}$, $T, S_0>0$, show that the map $\sigma\mapsto C^{\mathrm{BS}}(S_0, K, T;\sigma)$ is bijective;
- Compute 
$$
\lim_{\sigma\downarrow 0}C^{\mathrm{BS}}(S_0, K, T;\sigma)
$$
and 
$$
\lim_{\sigma\uparrow\infty}C^{\mathrm{BS}}(S_0, K, T;\sigma).
$$
- Write a Python function with inputs $S_0, K, T, C$ that returns the implied volatility, solution to the equation $C^{\mathrm{BS}}(S_0, K, T;\sigma) = C$.
*Hint: You can either use a root-finding method using the bijective property, or try to minimise the squared difference*.
- Verify that the implied volatility corresponding to $(S_0, K, T, C) = (100., 100., 1., 7.9655674)$ is equal to $20\%$.